## Laplacian Eigenmaps


Demonstration of using the method of [1] for representation learning on graphs. The node representations are used to perform node attribute inference on a paper citation network, namely Cora. 

**References**

[1] Laplacian Eigenmaps and Spectral Techniques for Embedding and Clustering, M. Belkin and P.Niyogi, NIPS 2002

Copyright 2010-2019 Commonwealth Scientific and Industrial Research Organisation (CSIRO).

All Rights Reserved.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import os
import networkx as nx
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

%matplotlib inline

## Dataset

### Dataset


The dataset is the citation network Cora.

It can be downloaded by clicking [here](https://linqs-data.soe.ucsc.edu/public/lbc/cora.tgz)

The following is the description of the dataset from the publisher,

> The Cora dataset consists of 2708 scientific publications classified into one of seven classes. The citation network consists of 5429 links. Each publication in the dataset is described by a 0/1-valued word vector indicating the absence/presence of the corresponding word from the dictionary. The dictionary consists of 1433 unique words. The README file in the dataset provides more details. 

For this demo, we ignore the word vectors associated with each paper. We are only interested in the network structure and the **subject** attribute of each paper.

Download and unzip the cora.tgz file to a location on your computer. 

We assume that the dataset is stored in the directory

`../data/cora/`

where the files `cora.cites` and `cora.content` can be located.

We are going to load the data into a networkx object.

### Specify the data input directory

**Note:** Make sure this is set correctly on your machine!

In [ ]:
data_dir = os.path.expanduser("../data/cora/")

In [ ]:
cora_location = os.path.expanduser(os.path.join(data_dir, "cora.cites"))
g_nx = nx.read_edgelist(path=cora_location)

# load the node attribute data
cora_data_location = os.path.expanduser(os.path.join(data_dir, "cora.content"))
node_attr = pd.read_csv(cora_data_location, sep='\t', header=None)
values = { str(row.tolist()[0]): row.tolist()[-1] for _, row in node_attr.iterrows()}
nx.set_node_attributes(g_nx, values, 'subject')

Load the features and subject for the nodes

In [ ]:
feature_names = ["w_{}".format(ii) for ii in range(1433)]
column_names =  feature_names + ["subject"]
node_data = pd.read_table(os.path.join(data_dir, "cora.content"), header=None, names=column_names)

We are going to use only the largest graph connected component.

In [ ]:
# Select the largest connected component. For clarity we ignore isolated
# nodes and subgraphs; having these in the data does not prevent the
# algorithm from running and producing valid results.
g_nx_ccs = (g_nx.subgraph(c).copy() for c in nx.connected_components(g_nx))
g_nx = max(g_nx_ccs, key=len)
print("Largest subgraph statistics: {} nodes, {} edges".format(
    g_nx.number_of_nodes(), g_nx.number_of_edges()))

Retrieve the labels for the nodes in the graph

In [ ]:
node_ids = list(g_nx.nodes())
node_ids[:10]

In [ ]:
node_targets = [ g_nx.nodes[node_id]['subject'] for node_id in node_ids]

In [ ]:
node_targets[:10]

In [ ]:
np.unique(node_targets) # should be 1 of 7 subjects

In [ ]:
len(node_targets)

In [ ]:
len(g_nx.nodes())

In [ ]:
y = node_targets
y[0:5]

Retrieve the feature vectors for the nodes in the largest connected component.

We won't use these at first but we might use them later

In [ ]:
node_data.drop(["subject"], inplace=True, axis=1)  # drop the subject column
node_data.index = node_data.index.map(str)  # make sure the index is string because the graph uses string for node ids

node_data = node_data[node_data.index.isin(list(g_nx.nodes()))]  # get the rows for the nodes in the graph

node_data = node_data.reindex(list(g_nx.nodes()))# reindex so that the order of the features is the same as the order of
                                                 # the nodes in the graph and the order of the nodes in the adjacency
                                                 # matrix

In [ ]:
node_data.head()

In [ ]:
# We are going to use these to draw the data such that nodes with the same subject have the
# same color.
colors = {'Case_Based': 'black',
          'Genetic_Algorithms': 'red',
          'Neural_Networks': 'blue',
          'Probabilistic_Methods': 'green',
          'Reinforcement_Learning': 'aqua',
          'Rule_Learning': 'purple',
          'Theory': 'yellow'}

### Calculate the graph Laplacian

There are 3 graph Laplacians commonly used. These are called unormalized, random walk and normalised graph Laplacian and they are defined as follows:

Unormalised: $L = D-A$

Random Walk: $L_{rw} = D^{-1}L = I - D^{-1}A$

Normalised:  $L_{sym} = D^{-1/2}LD^{-1/2} = I - D^{-1/2}AD^{-1/2}$

We are going to consider the unormalised graph Laplacian.

### First retrieve the adjacency matrix from the networkx object

In [ ]:
A = nx.to_numpy_array(g_nx)  # The graph adjacency matrix

### Calculate the degree matrix

In [ ]:
D = np.diag(A.sum(axis=1))  # sum rows

print(D)

### Calculate the Laplacian

In [ ]:
L = D-A  # The Graph Laplacian

In [ ]:
L

### Calculate the eigenvectors and corresponding eigenvalues

In [ ]:
embedding_size = 32

In [ ]:
w, v = np.linalg.eig(L)  # w is vector of eigenvalues
                         # v columns are eigenvectors

In [ ]:
w.shape

In [ ]:
v.shape

In [ ]:
# Just use the real parts of w and v
w = np.real(w)
v = np.real(v)

In [ ]:
order = np.argsort(w)  # from smallest to largest eigenvalue
w = w[order]
v_0 = v[:, order[0]]
v = v[:, order[1:(embedding_size+1)]]  # ignore the first one

In [ ]:
print(w)

In [ ]:
plt.plot(w)

In [ ]:
v_0

In [ ]:
print(v)

In [ ]:
tsne = TSNE(n_components=2)  # use PCA for speed
v_pr = tsne.fit_transform(v)

In [ ]:
v_pr.shape

In [ ]:
# draw the points
alpha=0.7
label_map = { l: i for i, l in enumerate(np.unique(node_targets))}
node_colours = [ label_map[target] for target in node_targets]

fig = plt.figure(figsize=(10,8))
plt.scatter(v_pr[:,0], 
            v_pr[:,1], 
            c=node_colours, cmap="jet", alpha=alpha)
# fig.savefig("LE32_embeddings.png")

### Train a Random Forest classifier

In [ ]:
X = v
Y = np.array(y)

In [ ]:
X.shape, Y.shape

### Some other suitable classifiers

Other than Random Forrest classification, one can use any of the following,

[Logistic Regression](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression)
[Support Vector Classification](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC)
[Nearest Neighbors](https://scikit-learn.org/stable/modules/neighbors.html#nearest-neighbors-classification)


In [ ]:
clf = RandomForestClassifier(n_estimators=10, min_samples_leaf=4)

X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=140, random_state=42)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
print("score on X_train {}".format(clf.score(X_train, y_train)))
print("score on X_test {}".format(clf.score(X_test, y_test)))